In [ ]:
!pip install miditok
# Installa le librerie necessarie
!pip install pandas numpy tqdm
!pip install music21 tensorflow


In [ ]:
import glob
import os
import music21
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import math
from pathlib import Path

# Import Keras/TensorFlow
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, Activation, BatchNormalization, Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import Sequence, to_categorical
from keras.regularizers import l2
from keras.optimizers import RMSprop
from keras import mixed_precision
import tensorflow as tf
# Import per il processing
from concurrent.futures import ProcessPoolExecutor
from sklearn.model_selection import train_test_split

# Import per la generazione
from music21 import converter, instrument, note, chord, stream


In [ ]:

data_dir = Path('data/maestro-v3.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v3.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

In [ ]:
df = pd.read_csv('/content/data/maestro-v3.0.0/maestro-v3.0.0.csv', sep=",")
autori = df.groupby("canonical_composer").size().reset_index(name="count").sort_values(by="count", ascending=False)
# Filtra il DataFrame 'autori' per mantenere solo le righe dove 'count' è > 10
autori_con_piu_di_10_brani = autori[autori['count'] > 10]

# Visualizza il risultato
print(autori_con_piu_di_10_brani)

In [ ]:
STILE_MAP = {
    'Johann Sebastian Bach': 'STYLE_BAROQUE', 'Domenico Scarlatti': 'STYLE_BAROQUE',
    'Ludwig van Beethoven': 'STYLE_CLASSICAL', 'Wolfgang Amadeus Mozart': 'STYLE_CLASSICAL',
    'Joseph Haydn': 'STYLE_CLASSICAL',
    'Frédéric Chopin': 'STYLE_ROMANTIC', 'Franz Schubert': 'STYLE_ROMANTIC'
}

# --- LA CORREZIONE CHIAVE È QUI ---
NUM_MIDI_FOR_COMPOSER_MAP = {
    'Johann Sebastian Bach': 120, 'Domenico Scarlatti': 31,
    'Ludwig van Beethoven': 80, 'Wolfgang Amadeus Mozart': 38,
    'Joseph Haydn': 40,
    'Frédéric Chopin': 80, 'Franz Schubert': 80
}

# Assicurati che i nomi qui corrispondano esattamente a quelli in 'canonical_composer' nel tuo CSV!
COMPOSITORI_SELEZIONATI = list(STILE_MAP.keys())

In [ ]:
LOCAL_PATH = Path('/content/data/maestro-v3.0.0')
# AGGIORNA I PERCORSI PER PUNTARLI AL TUO DRIVE
BASE_DIR = Path('/content/drive/MyDrive/maestro-project/LLM') # <-- ESEMPIO, MODIFICA QUI
METADATA_FILE = LOCAL_PATH / 'maestro-v3.0.0.csv'
OUTPUT_DIR = BASE_DIR # <-- ESEMPIO, MODIFICA QUI

# Aggiorna anche i percorsi di output globali
TOKENIZED_MIDI_SONGS_PATH = OUTPUT_DIR / "all_tokenized_songs.json"
TOKENIZER_SAVE_PATH = OUTPUT_DIR / "tokenizer.json"
TRAIN_BIN_PATH = OUTPUT_DIR / "train.bin"
VAL_BIN_PATH = OUTPUT_DIR / "val.bin"
META_PKL_PATH = OUTPUT_DIR / "meta.pkl"

In [ ]:
!cat /content/model.py

In [ ]:

import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from miditok import TokenizerConfig, REMI

METADATA_FILE = LOCAL_PATH / 'maestro-v3.0.0.csv'




In [ ]:
SPECIAL_TOKENS = ["PAD_TOKEN", "BOS_TOKEN", "EOS_TOKEN", "MASK_TOKEN"] + list(set(STILE_MAP.values()))
print(SPECIAL_TOKENS)
TOKENIZER_CONFIGV2 = TokenizerConfig(special_tokens=SPECIAL_TOKENS)
tokenizer = REMI(tokenizer_config=TOKENIZER_CONFIGV2)

In [ ]:
def midi_filter_tokenizzation():
  metadata = pd.read_csv(METADATA_FILE)
  metadata_filtrati = metadata[metadata['canonical_composer'].isin(STILE_MAP.keys())].copy()
  metadata_filtrati['style'] = metadata_filtrati['canonical_composer'].map(STILE_MAP)
  print(metadata_filtrati.head())

  metadata_campionati = metadata_filtrati.groupby('canonical_composer').apply(
      lambda group: group.sample(
          n = min(NUM_MIDI_FOR_COMPOSER_MAP[group.name], len(group)),
          random_state = 42
      ))
   # --- VERIFICA (Come prima) ---
  print("Numero di brani per compositore dopo il campionamento casuale:")
  print(metadata_campionati['canonical_composer'].value_counts())
  print("-" * 30)

  file_midi_da_processare = metadata_campionati['midi_filename'].tolist()
  map_midi2style = metadata_campionati.set_index("midi_filename")["style"].to_dict()
  all_tokenized_song = []
  print(f"🎼 Trovati {len(file_midi_da_processare)} file per i compositori: {', '.join(COMPOSITORI_SELEZIONATI)}")
  if not file_midi_da_processare:
    return None
  print(file_midi_da_processare)
  for midi_filename in tqdm(file_midi_da_processare):
    file_path = LOCAL_PATH / midi_filename
    tokens = tokenizer(file_path)

    full_seq =[tokenizer.vocab["BOS_TOKEN"]] + [tokenizer.vocab[map_midi2style[midi_filename]]] + tokens[0].ids + [tokenizer.vocab["EOS_TOKEN"]]
    all_tokenized_song.append(full_seq)
  return all_tokenized_song



In [ ]:
def midi_tokenizzation(tokenizer):
  metadata = pd.read_csv(METADATA_FILE)


  file_midi_da_processare = metadata['midi_filename'].tolist()


  all_tokenized_song = []
  print(f"🎼 Trovati {len(file_midi_da_processare)} file MIDI totali nel dataset.")
  if not file_midi_da_processare:
    return None

  # Aggiungi la barra di progresso per i file MIDI
  for midi_filename in tqdm(file_midi_da_processare, desc="Tokenizing MIDI files"):
    file_path = LOCAL_PATH / midi_filename
    try:
        tokens = tokenizer(file_path)

        if tokens and tokens[0].ids: # Controlla se tokens e tokens[0].ids non sono vuoti
            full_seq = [tokenizer.vocab["BOS_TOKEN"]] + tokens[0].ids + [tokenizer.vocab["EOS_TOKEN"]]
            all_tokenized_song.append(full_seq)
        else:
            print(f"⚠️ Skipping {midi_filename}: Could not tokenize or empty sequence.")


    except Exception as e:
        print(f"❌ Errore durante la tokenizzazione di {midi_filename}: {e}")
        continue # Continua con il prossimo file in caso di errore

  print(f"✅ Tokenizzazione completata. Elaborati {len(all_tokenized_song)} brani.")
  return all_tokenized_song

In [ ]:
all_tokenized_song = midi_filter_tokenizzation()

In [ ]:
import json
def save_tokenized_midi(file_path, all_tokenized_song):

  print(f"💾 Salvataggio di {len(all_tokenized_song)} brani tokenizzati in {file_path}...")
  with open(file_path, 'w') as f:
    json.dump(all_tokenized_song, f)
  print("Salvataggio completato!")

def load_tokenized_midi(file_path):

  print(f"📂 Caricamento dei brani tokenizzati da {file_path}...")

  with open(file_path, 'r') as f:
    loaded_tokenized_songs = json.load(f)

  print(f"Caricati {len(loaded_tokenized_songs)} brani.")
  return loaded_tokenized_songs


In [ ]:
import random
songs = all_tokenized_song
# Shuffle i brani per non avere bias di ordine
random.shuffle(songs)

n = len(songs)
train_split = int(0.8 * n)
val_split = int(0.9 * n)

train_songs = songs[:train_split]
val_songs = songs[train_split:val_split]
test_songs = songs[val_split:]

def flatten(song_list):
    tokens = []
    for song in song_list:
        tokens.extend(song)
    return tokens

all_tokens_train = flatten(train_songs)
all_tokens_val = flatten(val_songs)
all_tokens_test = flatten(test_songs)

In [ ]:
import torch
from torch.utils.data import Dataset

class MidiDataset(Dataset):
    def __init__(self, tokens, seq_len=512):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len

    def __getitem__(self, idx):
        x = self.tokens[idx : idx + self.seq_len]
        y = self.tokens[idx + 1 : idx + self.seq_len + 1]
        return torch.tensor(x), torch.tensor(y)


In [ ]:
train_dataset = MidiDataset(all_tokens_train, seq_len=1024)
val_dataset   = MidiDataset(all_tokens_val, seq_len=1024)
test_dataset  = MidiDataset(all_tokens_test, seq_len=1024)

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE =  256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)


In [ ]:
tokenizer.vocab

In [ ]:
from model import GPT, GPTConfig
config = GPTConfig(
    block_size=1024,      # Lunghezza massima della sequenza di input
    vocab_size=tokenizer.vocab_size,    # Dimensione del vocabolario
    n_layer=6,           # Numero di blocchi Transformer
    n_head=6,            # Numero di "teste" di attenzione
    n_embd=96,           # Dimensione dei vettori di embedding
    dropout=0.1,
    bias = False
)


model = GPT(config)

In [ ]:
model = torch.compile(model)

# Ottimizzatore AdamW
learning_rate = 1e-3
max_iters = 5000
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95



In [ ]:
import torch
import time
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from pathlib import Path

# --- CONFIGURAZIONE ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_float32_matmul_precision('high')  # Tensor Cores

# Hyperparametri
learning_rate = 5e-4
weight_decay = 0.1
beta1, beta2 = 0.9, 0.95
max_epochs = 10
batch_size = 512              # batch sicuro
grad_accum_steps = 2          # simula batch 512
grad_clip_norm = 1.0
eval_interval = 1

# --- DATALOADERS ---
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=4, pin_memory=True)

# --- MODELLO ---
model.to(device)
print(f"Modello spostato su: {device}")

# --- OTTIMIZZATORE ---
optimizer = model.configure_optimizers(weight_decay=weight_decay,
                                       learning_rate=learning_rate,
                                       betas=(beta1, beta2),
                                       device_type=device)

# --- SCHEDULER LR ---
total_steps = max_epochs * len(train_loader) // grad_accum_steps
warmup_steps = max(50, int(0.01 * total_steps))

def lr_lambda(current_step):
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    progress = float(current_step - warmup_steps) / float(max(1, total_steps - warmup_steps))
    return 0.5 * (1.0 + np.cos(np.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# --- Mixed precision scaler ---
scaler = torch.cuda.amp.GradScaler()

# --- FUNZIONE DI STIMA LOSS ---
@torch.no_grad()
def estimate_metrics(): # Rinominata per chiarezza
    metrics = {}
    model.eval()
    for split_name, loader in [('train', train_loader), ('val', val_loader)]:
        losses = []
        correct_predictions = 0
        total_samples = 0

        for i, (X, Y) in enumerate(loader):
            if i >= 50:  # limita valutazione per velocità
                break
            X, Y = X.to(device, non_blocking=True), Y.to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                logits, loss = model(X, Y)

            losses.append(loss.item())

            # Calcolo dell'accuracy
            preds = torch.argmax(logits, dim=2) # Changed dim from 1 to 2
            correct_predictions += (preds.view(-1) == Y.view(-1)).sum().item() # Flatten tensors before comparison
            total_samples += Y.numel() # Use numel() for total number of elements

        metrics[f'{split_name}_loss'] = np.mean(losses)
        metrics[f'{split_name}_accuracy'] = correct_predictions / total_samples

    model.train()
    return metrics

# --- CHECKPOINT ---
best_val_loss = float('inf')
checkpoint_path = OUTPUT_DIR / 'CheckPoint'
checkpoint_path.mkdir(parents=True, exist_ok=True) # Create the directory if it doesn't exist
checkpoint_file = checkpoint_path / 'best_model.pt' # Define the checkpoint file name

# --- TRAINING LOOP ---
print("🚀 Inizio dell'addestramento...")
start_time = time.time()
step_timer = start_time
global_step = 0
optimizer.zero_grad(set_to_none=True)

for epoch in range(max_epochs):
    model.train()
    for i, (X, Y) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{max_epochs}")):
        X, Y = X.to(device, non_blocking=True), Y.to(device, non_blocking=True)

        with torch.cuda.amp.autocast():
            logits, loss = model(X, Y)
            loss = loss / grad_accum_steps  # scala per accumulation

        scaler.scale(loss).backward()

        # Gradient accumulation
        if (i + 1) % grad_accum_steps == 0 or (i + 1) == len(train_loader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

            # Logging step/sec e VRAM
            if global_step % 10 == 0:
                elapsed = time.time() - step_timer
                step_timer = time.time()
                mem_alloc = torch.cuda.memory_allocated() / 1024**2
                mem_total = torch.cuda.get_device_properties(device).total_memory / 1024**2
                current_lr = scheduler.get_last_lr()[0]
                print(f"Step {global_step}, {10/elapsed:.2f} step/sec, VRAM {mem_alloc:.1f}/{mem_total:.1f} MB, LR {current_lr:.2e}")

    # Valutazione periodica
    if epoch % eval_interval == 0 or epoch == max_epochs - 1:
        metrics = estimate_metrics()
        current_val_loss = metrics['val_loss']
        elapsed_time = time.time() - start_time
        print("-" * 70)
        print(f"Epoch {epoch+1}: train loss {metrics['train_loss']:.4f}, val loss {current_val_loss:.4f}, "
              f"train acc {metrics['train_accuracy']:.2%}, val acc {metrics['val_accuracy']:.2%}, "
              f"tempo: {elapsed_time:.2f}s")
        print("-" * 70)

        # Checkpoint se migliora
        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            print(f"🎉 Nuova val_loss migliore: {best_val_loss:.4f}. Salvataggio checkpoint...")
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'best_val_loss': best_val_loss,
                'config': config,
                'tokenizer_vocab': tokenizer.vocab
            }
            torch.save(checkpoint, checkpoint_file) # Use checkpoint_file for saving
        else:
            print(f"Val_loss ({current_val_loss:.4f}) non migliorata rispetto a {best_val_loss:.4f}.")
        print("-" * 50)

print("✅ Addestramento completato.")

In [ ]:
# Salva il file MIDI
output_midi_path = OUTPUT_DIR / "generated_composition.mid"
generated_midi.dump_midi(output_midi_path)

print(f"\nComposizione generata e salvata in: {output_midi_path}")

In [ ]:
import torch
from pathlib import Path

# Definisci il percorso dove salvare il modello
# Puoi modificarlo se vuoi salvare in un'altra posizione o con un nome diverso
model_save_path = OUTPUT_DIR / "final_model_state_dict.pt"

# Assicurati che la directory di destinazione esista
model_save_path.parent.mkdir(parents=True, exist_ok=True)

# Salva lo stato del modello
# Stiamo salvando solo lo state_dict, che è sufficiente per ricaricare il modello
torch.save(model.state_dict(), model_save_path)

print(f"Stato del modello salvato in: {model_save_path}")

In [ ]:
import torch
import torch.nn.functional as F

def generate_midi(model, start_token, max_len=1024, temperature=1.0, top_k=None, device='cpu'):
    """
    Genera una sequenza di token MIDI a partire da start_token.

    Args:
        model: il modello Transformer già allenato
        start_token: intero del token iniziale (<BOS>)
        max_len: lunghezza massima della sequenza generata
        temperature: controlla casualità (1.0 = normale, <1 più conservativo, >1 più creativo)
        top_k: se non None, prende solo i k token più probabili a ogni step
        device: 'cpu' o 'cuda'

    Returns:
        lista di token generati (incluso start_token e )"""
    model.eval()
    sequence = [start_token]
    with torch.no_grad():
        for _ in range(max_len - 1):
            input_tensor = torch.tensor(sequence).unsqueeze(0).to(device)
            output = model(input_tensor)
            # Estrai i logit dall'output del modello. Se l'output è una tupla, prendi il primo elemento.
            logits = output[0] if isinstance(output, tuple) else output

            # Prendi i logit solo per l'ultimo token generato
            logits = logits[:, -1, :] / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float('Inf')

            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).squeeze(1).tolist()[0]
            sequence.append(next_token)

            # Interrompi se viene generato il token EOS
            # if next_token == <EOS_TOKEN_VALUE>: # Sostituisci con il valore effettivo del token EOS
            #     break

    return sequence

#Gen

In [ ]:
!sudo apt-get install -y fluidsynth
!wget https://github.com/FluidSynth/fluidsynth/raw/master/sf2/FluidR3_GM.sf2
!pip install midi2audio

In [ ]:
checkpoint_gen = torch.load("/content/drive/MyDrive/maestro-project/LLM/CheckPoint/best_model.pt", weights_only=False)

In [ ]:
import torch
from collections import OrderedDict
unwanted_prefix = '_orig_mod.'
state_dict = checkpoint_gen

# Crea un nuovo dizionario senza il prefisso
clean_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith(unwanted_prefix):
        # Rimuovi il prefisso
        new_k = k[len(unwanted_prefix):]
        clean_state_dict[new_k] = v
    else:
        # Se qualche chiave non ha il prefisso, tienila com'è
        clean_state_dict[k] = v

In [ ]:
state_dict_loaded = checkpoint_gen['model_state_dict']

# 5. Crea un nuovo dizionario e rimuovi il prefisso '_orig_mod.' da ogni chiave
new_state_dict = OrderedDict()
for k, v in state_dict_loaded.items():
    # Usa .removeprefix() per un codice più pulito e moderno (Python 3.9+)
    # Se usi una versione più vecchia di Python, usa: name = k[len('_orig_mod.'):]
    name = k.removeprefix('_orig_mod.')
    new_state_dict[name] = v

# 6. Carica il state_dict corretto e pulito nel tuo modello
model.load_state_dict(new_state_dict)

print("Modello caricato con successo!")

In [ ]:
model.eval()

# Sposta il modello sulla GPU se disponibile
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(f"Modello pronto per l'inferenza su dispositivo: {device}")

In [ ]:
prompt_tokens = val_dataset[0][:100] # Prende le etichette (Y) dal primo elemento
prompt_tokens = prompt_tokens[0][:1].tolist() + [6] + prompt_tokens[0][1:].tolist()
print(prompt_tokens)

In [ ]:

song = []
for midi_filename in tqdm('/content/data/maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_02_Track02_wav.midi'):
    file_path = '/content/data/maestro-v3.0.0/2009/MIDI-Unprocessed_20_R1_2009_01-05_ORIG_MID--AUDIO_20_R1_2009_20_R1_2009_02_WAV.midi'
    tokens = tokenizer(file_path)

    full_seq =[tokenizer.vocab["BOS_TOKEN"]] + [5] + tokens[0].ids + [tokenizer.vocab["EOS_TOKEN"]]
    song.append(full_seq)

In [ ]:
import numpy as np
#song = np.random.randint(low=9, high=288, size=1024)
# 2. Convertila in un tensore
mio_tensore = torch.tensor(song[:1024])

print(f"La mia lista: {song}")
print(f"Il mio tensore: {len(mio_tensore)}")
print(f"Tipo di dato del tensore: {mio_tensore.dtype}")

In [ ]:
prompt_tokens = val_dataset[0][:300] # Prende le etichette (Y) dal primo elemento
prompt_tokens = mio_tensore
# Converti in un tensore di PyTorch con la dimensione del batch (1)
# Seleziona il primo elemento della tupla prompt_tokens
prompt_tensor = torch.tensor(prompt_tokens[0], dtype=torch.long, device=device).unsqueeze(0)


print("\nPrompt iniziale (decodificato):")
# Wrap the list of token IDs in another list
print(tokenizer.decode([prompt_tensor.squeeze(0).tolist()]))
print("-" * 50)
print("Inizio generazione...\n")

# --- GENERAZIONE AUTOREGRESSIVA ---
@torch.no_grad() # Disabilita il calcolo dei gradienti per l'inferenza
def generate(model, idx, max_new_tokens, temperature=1.0, top_k=None):
    """
    Funzione di generazione di testo.
    idx: tensore (B, T) di indici di token nel contesto attuale
    """
    for _ in range(max_new_tokens):
        # Se il contesto diventa troppo lungo, taglialo per adattarlo alla block_size del modello
        block_size = model.config.block_size # Assumendo che la config abbia block_size
        idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]

        # Forward pass per ottenere i logits per l'ultimo token
        logits, _ = model(idx_cond) # Ignoriamo la loss durante l'inferenza
        logits = logits[:, -1, :] # Prendi solo i logits per l'ultimo step temporale -> (B, vocab_size)

        # Applica la temperatura
        logits = logits / temperature

        # (Opzionale) Applica Top-k sampling
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf') # Metti a -infinito i logits non nel top-k

        # Converti i logits in probabilità
        probs = torch.nn.functional.softmax(logits, dim=-1)

        # Campiona il token successivo dalla distribuzione di probabilità
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

        # Aggiungi il nuovo token alla sequenza
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

# Esegui la generazione
# Definisci max_new_tokens, temperature e top_k (puoi usare i valori che preferisci)
max_new_tokens = 1024  # Esempio: genera 200 nuovi token
temperature = 0.9     # Esempio: temperatura per il campionamento
top_k = 20            # Esempio: usa top-k sampling con k=50

generated_tokens_tensor = generate(model, prompt_tensor, max_new_tokens, temperature, top_k)

# Estrai solo i tokens generati (escludendo il prompt)
generated_ids = generated_tokens_tensor.squeeze(0).tolist()[len(prompt_tokens):] # Adjusted index here

print("Token ID generati:", generated_ids) # Print generated token IDs

# Filter out tokens outside the vocabulary size
vocab_size = len(tokenizer.vocab)
filtered_generated_ids = [t for t in generated_ids if t < vocab_size]

# Wrap the list of filtered token IDs in another list and decode
generated_text = tokenizer.decode([filtered_generated_ids])

# --- STAMPA IL RISULTATO ---
print("Testo generato:")
print(generated_text)
print("-" * 50)



In [ ]:
# Assign filtered_generated_ids to generated_ids for the next cell (Cg0D20LH0hyH)
generated_ids = filtered_generated_ids

midi_obj = tokenizer.decode([generated_ids]) # Use the filtered generated_ids
midi_obj.dump_midi('/content/sample_data/mus.mid')

In [ ]:
from midi2audio import FluidSynth
from IPython.display import Audio

# Chuyển MIDI sang WAV
fs = FluidSynth()
fs.midi_to_audio('/content/sample_data/mus.mid', 'output.wav')

# Phát WAV file
Audio('output.wav')

In [ ]:
from google.colab import files

# Scarica il file WAV generato
files.download('output.wav')